In [1]:
import pandas as pd
import numpy as np

In [11]:
init_emissions = pd.read_csv("emissions-Table 1.csv", index_col=0).iloc[:, :3]

In [16]:
init_emissions.T

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
Helix,11.9464,5.8,3.7048,5.4004,1.0710,4.5632,8.6060,4.8794,2.0944,5.2416,11.2644,6.2263,2.4,3.8,2.2278,5.2110,4.6872,1.3857,3.2806,6.2098
sheet,6.6272,4.6,2.6102,3.1698,1.5498,2.7968,4.6340,5.1942,2.3800,9.6096,10.3704,4.4080,2.0,5.6,2.0382,4.8636,6.6960,1.8923,4.9558,14.0041
other,6.8016,4.4,5.5993,7.4549,1.1592,3.2752,5.6932,12.4346,2.5704,3.4944,5.9898,5.5100,1.6,3.2,8.6742,6.7164,5.9706,1.1920,2.7920,5.4722


In [17]:
print(init_emissions.T.to_numpy())

[[11.9464  5.8     3.7048  5.4004  1.071   4.5632  8.606   4.8794  2.0944
   5.2416 11.2644  6.2263  2.4     3.8     2.2278  5.211   4.6872  1.3857
   3.2806  6.2098]
 [ 6.6272  4.6     2.6102  3.1698  1.5498  2.7968  4.634   5.1942  2.38
   9.6096 10.3704  4.408   2.      5.6     2.0382  4.8636  6.696   1.8923
   4.9558 14.0041]
 [ 6.8016  4.4     5.5993  7.4549  1.1592  3.2752  5.6932 12.4346  2.5704
   3.4944  5.9898  5.51    1.6     3.2     8.6742  6.7164  5.9706  1.192
   2.792   5.4722]]


,Helix,sheet,other
A,11.9464,6.6272,6.8016
R,5.8000,4.6000,4.4000
N,3.7048,2.6102,5.5993
D,5.4004,3.1698,7.4549
C,1.0710,1.5498,1.1592
Q,4.5632,2.7968,3.2752
E,8.6060,4.6340,5.6932
G,4.8794,5.1942,12.4346
H,2.0944,2.3800,2.5704
I,5.2416,9.6096,3.4944
